# Running Jobs on Hadoop

The aim of this exercise is to become familiar with Hadoop. We will show how to run Hadoop application and create our own Python-based streaming application for parsing log file data.

## Configure Hadoop Environment and Test Applications

For the following exercise we will use two examples provided as part of the standard Hadoop distribution. We use the Hortonworks HDP 2.3.2 deployed on Amazon Web Services (EC2). First we need to set these two variables to the `jar` files containing the application.

In [37]:
HADOOP_EXAMPLES="/usr/hdp/2.3.2.0-2950/hadoop-mapreduce/hadoop-mapreduce-examples.jar"
HADOOP_STREAMING="/usr/hdp/2.3.2.0-2950/hadoop-mapreduce/hadoop-streaming.jar"

We will use the standard Hadoop command line utilities to enquire the status of the cluster (in particular HDFS and Yarn), and to submit applications. Shell commands can be executed in iPython notebook cells using the prefix `!`. Use shift-enter or the play button in the menu to execute cells. 

## 1. Hadoop Services (HDFS and YARN)

The `-report` argument provides information about the file-system, disk space, number of nodes etc.

In [32]:
!hadoop dfsadmin -report

DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

Configured Capacity: 638810435584 (594.94 GB)
Present Capacity: 572727713792 (533.39 GB)
DFS Remaining: 504732938240 (470.07 GB)
DFS Used: 67994775552 (63.33 GB)
DFS Used%: 11.87%
Under replicated blocks: 4
Blocks with corrupt replicas: 0
Missing blocks: 0
Missing blocks (with replication factor 1): 0

-------------------------------------------------
report: Access denied for user radical. Superuser privilege is required


In [33]:
!yarn node -list all

15/11/15 15:38:14 INFO impl.TimelineClientImpl: Timeline service address: http://ip-10-63-179-69.ec2.internal:8188/ws/v1/timeline/
15/11/15 15:38:14 INFO client.RMProxy: Connecting to ResourceManager at ip-10-63-179-69.ec2.internal/10.63.179.69:8050
Total Nodes:6
         Node-Id	     Node-State	Node-Http-Address	Number-of-Running-Containers
ip-10-63-179-69.ec2.internal:45454	        RUNNING	ip-10-63-179-69.ec2.internal:8042	                           0
ip-10-47-179-69.ec2.internal:45454	        RUNNING	ip-10-47-179-69.ec2.internal:8042	                           0
ip-10-145-0-4.ec2.internal:45454	        RUNNING	ip-10-145-0-4.ec2.internal:8042	                           0
ip-10-99-194-113.ec2.internal:45454	        RUNNING	ip-10-99-194-113.ec2.internal:8042	                           0
ip-10-218-164-206.ec2.internal:45454	        RUNNING	ip-10-218-164-206.ec2.internal:8042	                           0
ip-10-179-174-236.ec2.internal:45454	        RUNNING	ip-10-179-174-236.ec2.internal:

## 2. Terasort

The Terasort application consists of two parts: (i) data generation (`teragen`), and (ii) the sorting of the data (`terasort`). 

Note that if you execute `teragen` multiple times, please make sure to delete the target directory (`teragen`); otherwise an error is thrown as shown.

In [34]:
!hdfs dfs -rm -r teragen teraout

15/11/15 15:38:45 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 360 minutes, Emptier interval = 0 minutes.
Moved: 'hdfs://ip-10-63-179-69.ec2.internal:8020/user/radical/teragen' to trash at: hdfs://ip-10-63-179-69.ec2.internal:8020/user/radical/.Trash/Current
15/11/15 15:38:45 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 360 minutes, Emptier interval = 0 minutes.
Moved: 'hdfs://ip-10-63-179-69.ec2.internal:8020/user/radical/teraout' to trash at: hdfs://ip-10-63-179-69.ec2.internal:8020/user/radical/.Trash/Current


In the next command we will create a dataset of 100,000 records,  each of 100 bytes (for a total of 10Mb).

In [35]:
!yarn jar $HADOOP_EXAMPLES teragen 100000 teragen

15/11/15 15:39:29 INFO impl.TimelineClientImpl: Timeline service address: http://ip-10-63-179-69.ec2.internal:8188/ws/v1/timeline/
15/11/15 15:39:29 INFO client.RMProxy: Connecting to ResourceManager at ip-10-63-179-69.ec2.internal/10.63.179.69:8050
15/11/15 15:39:30 INFO terasort.TeraSort: Generating 100000 using 2
15/11/15 15:39:30 INFO mapreduce.JobSubmitter: number of splits:2
15/11/15 15:39:30 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1447000128355_0778
15/11/15 15:39:31 INFO impl.YarnClientImpl: Submitted application application_1447000128355_0778
15/11/15 15:39:31 INFO mapreduce.Job: The url to track the job: http://ip-10-63-179-69.ec2.internal:8088/proxy/application_1447000128355_0778/
15/11/15 15:39:31 INFO mapreduce.Job: Running job: job_1447000128355_0778
15/11/15 15:39:37 INFO mapreduce.Job: Job job_1447000128355_0778 running in uber mode : false
15/11/15 15:39:37 INFO mapreduce.Job:  map 0% reduce 0%
15/11/15 15:39:43 INFO mapreduce.Job:  map 100% reduce 

---

The following command will use Hadoop commands (`yarn`) to perform sorting.

In [40]:
!yarn jar $HADOOP_EXAMPLES terasort teragen teraout

15/11/15 15:45:22 INFO terasort.TeraSort: starting
15/11/15 15:45:23 INFO input.FileInputFormat: Total input paths to process : 2
Spent 126ms computing base-splits.
Spent 2ms computing TeraScheduler splits.
Computing input splits took 129ms
Sampling 2 splits of 2
Making 1 from 100000 sampled records
Computing parititions took 358ms
Spent 490ms computing partitions.
15/11/15 15:45:24 INFO impl.TimelineClientImpl: Timeline service address: http://ip-10-63-179-69.ec2.internal:8188/ws/v1/timeline/
15/11/15 15:45:24 INFO client.RMProxy: Connecting to ResourceManager at ip-10-63-179-69.ec2.internal/10.63.179.69:8050
org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory hdfs://ip-10-63-179-69.ec2.internal:8020/user/radical/teraout already exists
	at org.apache.hadoop.examples.terasort.TeraOutputFormat.checkOutputSpecs(TeraOutputFormat.java:114)
	at org.apache.hadoop.mapreduce.JobSubmitter.checkSpecs(JobSubmitter.java:266)
	at org.apache.hadoop.mapreduce.JobSubmitter.submitJobI

In [39]:
!hadoop fs -text teraout/part-r-00000 | head

---

## 3. Word Count

Here we will discuss the `hello world` of Big Data: word count! We will count the words contained in the log file located at `/data/nasa/NASA_access_log_Jul95`

In [41]:
!hdfs dfs -rm -r wordcount-out

15/11/15 15:46:38 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 360 minutes, Emptier interval = 0 minutes.
Moved: 'hdfs://ip-10-63-179-69.ec2.internal:8020/user/radical/wordcount-out' to trash at: hdfs://ip-10-63-179-69.ec2.internal:8020/user/radical/.Trash/Current


In [42]:
!hdfs dfs -text /data/nasa/NASA_access_log_Jul95 | head

199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245
unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985
199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085
burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0
199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0
205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985
d104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985
129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] "GET / HTTP/1.0" 

In [43]:
!yarn jar $HADOOP_EXAMPLES wordcount /data/nasa/ wordcount-out/

15/11/15 15:47:07 INFO impl.TimelineClientImpl: Timeline service address: http://ip-10-63-179-69.ec2.internal:8188/ws/v1/timeline/
15/11/15 15:47:07 INFO client.RMProxy: Connecting to ResourceManager at ip-10-63-179-69.ec2.internal/10.63.179.69:8050
15/11/15 15:47:08 INFO input.FileInputFormat: Total input paths to process : 1
15/11/15 15:47:08 INFO mapreduce.JobSubmitter: number of splits:2
15/11/15 15:47:09 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1447000128355_0807
15/11/15 15:47:09 INFO impl.YarnClientImpl: Submitted application application_1447000128355_0807
15/11/15 15:47:09 INFO mapreduce.Job: The url to track the job: http://ip-10-63-179-69.ec2.internal:8088/proxy/application_1447000128355_0807/
15/11/15 15:47:09 INFO mapreduce.Job: Running job: job_1447000128355_0807
15/11/15 15:47:15 INFO mapreduce.Job: Job job_1447000128355_0807 running in uber mode : false
15/11/15 15:47:15 INFO mapreduce.Job:  map 0% reduce 0%
15/11/15 15:47:27 INFO mapreduce.Job:  map 3

## 4. Log Parsing

Use the commands `head`, `cat`, `uniq`, `wc`, `sort`, `find`, `xargs`, `awk` to evaluate the NASA log file:

Which page was called the most?
What was the most frequent return code?
How many errors occurred? What is the percentage of errors? Implement a Python version of this Unix Shell script using the script as a template (the answer can be found  in [mapreduce_streaming.py](mapreduce_streaming.py))

We will now run the Python script inside a Hadoop Streaming job.

In [28]:
!cat mapreduce_streaming.py 

#!/usr/bin/python
#
# Licensed to Cloudera, Inc. under one
# or more contributor license agreements.  See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership.  Cloudera, Inc. licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License.  You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
#
# Template for python Hadoop streaming.  Fill in the map() and reduce()
# functions, which should call emit(), as appropriate.
#
# Test your script with
#  cat input | python map_reduce.py map | sort | pytho

---
In the next example, we'll execute `mapreduce_streaming.py` with YARN as a hadoop streaming application.

In [29]:
!yarn jar $HADOOP_STREAMING -input /data/nasa -output logs-parsed \
                            -file mapreduce_streaming.py \
                            -mapper "python mapreduce_streaming.py map" \
                            -reducer "python mapreduce_streaming.py reduce" 

15/11/15 03:35:20 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapreduce_streaming.py] [/usr/hdp/2.3.2.0-2950/hadoop-mapreduce/hadoop-streaming-2.7.1.2.3.2.0-2950.jar] /var/lib/ambari-agent/tmp/hadoop_java_io_tmpdir/streamjob2404468226140792590.jar tmpDir=null
15/11/15 03:35:21 INFO impl.TimelineClientImpl: Timeline service address: http://ip-10-63-179-69.ec2.internal:8188/ws/v1/timeline/
15/11/15 03:35:21 INFO client.RMProxy: Connecting to ResourceManager at ip-10-63-179-69.ec2.internal/10.63.179.69:8050
15/11/15 03:35:22 INFO impl.TimelineClientImpl: Timeline service address: http://ip-10-63-179-69.ec2.internal:8188/ws/v1/timeline/
15/11/15 03:35:22 INFO client.RMProxy: Connecting to ResourceManager at ip-10-63-179-69.ec2.internal/10.63.179.69:8050
15/11/15 03:35:22 ERROR streaming.StreamJob: Error Launching job : Output directory hdfs://ip-10-63-179-69.ec2.internal:8020/user/radical/logs-parsed already exists
Streami

---
In the next two commands, we'll parse the output of the operations above.

In [30]:
!hdfs dfs -ls logs-parsed

Found 2 items
-rw-r--r--   3 radical hdfs          0 2015-11-08 16:48 logs-parsed/_SUCCESS
-rw-r--r--   3 radical hdfs         70 2015-11-08 16:48 logs-parsed/part-00000


In [44]:
!hdfs dfs -text logs-parsed/*

200	1701534
302	46573
304	132627
400	5
403	54
404	10845
500	62
501	14
